In [1]:
from transformers import pipeline, set_seed
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
from tqdm import tqdm
import torch

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/T0_3B")
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0_3B")

Maybe helpful if you use CUDA 11.3
```shell
pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
```
More versions: https://pytorch.org/get-started/previous-versions/

In [3]:
device = 'cpu'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
model = model.to(device)

cuda:0


RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 2.00 GiB total capacity; 1.73 GiB already allocated; 0 bytes free; 1.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Change the summaries here

In [3]:
summaries = pd.read_csv('../data/datasets/summary_title_tom_scott/tom_scott_title_summary.csv')
summaries

,id,title,summary
0,0,Reopening an airport terminal is harder than y...,The South Terminal at London's Gatwick Airport...
1,1,14 science fiction stories in under 6 minutes,I used to make short science fictionvideos for...
2,2,"After 140 years, this old technology still kee...","140 years ago, the Callander and Oban railway ..."
3,3,"The Elie Chainwalk is safe, as long as you fol...","The Elie Chainwalk is 500 metres long, made up..."
4,4,This town forgot to be a city,Rochester had been a city since the 13th centu...
...,...,...,...
236,236,Why California's Musical Road Sounds Terrible,If you drive down a certain stretch of highway...
237,237,The Reaction Ferries of Basel: What Have We Mi...,"CNN's John Sutter travels to Basel, Switzerlan..."
238,238,The Centuries-Old Debt That's Still Paying Int...,"In 1648, the Water Board of Lekdijk Bovendams ..."
239,239,Rotary Jails and Accidental Amputations,In 1881 an architect called William Brown and ...


In [7]:
generated_titles = list()
for _, row in tqdm(summaries.iterrows(), total=summaries.shape[0]):
    prompt = 'Write a title for that text: ' + row['summary']

    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(inputs.to(device), max_length=50)

    title = tokenizer.decode(outputs[0]).replace('<pad> ', '').replace('</s>', '')

    print(title)

    generated_titles.append(title)

  0%|          | 1/241 [00:30<2:02:30, 30.63s/it]

The South Terminal at Gatwick Airport is to reopen to the public on Sunday morning


  1%|          | 2/241 [00:57<1:54:13, 28.67s/it]

I'm not great at science fiction, but I have some ideas that I want to get


  1%|          | 2/241 [01:05<2:10:26, 32.75s/it]


KeyboardInterrupt: 

In [5]:
generated_titles

[' The South Terminal at Gatwick Airport is to reopen to the public on Sunday morning.']

In [ ]:
summaries['t0_3B_vanilla_title_summarized'] = generated_titles
summaries.to_csv('../data/generated/tom_scott_videos_t0_3B_vanilla_summarized_v2.csv')